In [1]:
import numpy as np
import pandas as pd
# import modin.pandas as pd

In [2]:
feature_path ='./feature/'
res_path = './res/'
data_path = './data/'
second_round_path = './second round/'

In [3]:
transaction_df = pd.read_csv(data_path+'transaction_train_new.csv')
operation_df =  pd.read_csv(data_path+'operation_train_new.csv')
label = pd.read_csv(data_path+'tag_train_new.csv')

transaction_test = pd.read_csv(data_path+'transaction_round1_new.csv')
operation_test = pd.read_csv(data_path+'operation_round1_new.csv')

# transaction_test = pd.read_csv(second_round_path+'test_transaction_round2.csv')
# operation_test = pd.read_csv(second_round_path+'test_operation_round2.csv')

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3044: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# Delete abnormal data
abnormal_device1 = [name for name, count in operation_df.groupby('device1').size().iteritems() if count > 20000]

# Stats Features


In [5]:
# operation_df = operation_df[~operation_df.device1.isin(abnormal_device1)]
# transaction_df = transaction_df[~transaction_df.device1.isin(abnormal_device1)]


In [6]:
def merge_count(df1,df2,columns,value,cname):
    add = df1.groupby(columns)[value].count().reset_index().rename(columns = {value:cname})
    df2=df2.merge(add,on=columns,how="left")
    del add
    return df2

def merge_nunique(df1,df2,columns,value,cname):
    add = df1.groupby(columns)[value].nunique().reset_index().rename(columns = {value:cname})
    df2=df2.merge(add,on=columns,how="left")
    del add
    return df2

def merge_notnull_count(df1,df2,columns,value,cname):
    add = df1[df1[value].notnull()].groupby(columns)[value].count().reset_index().rename(columns = {value:cname})
    df2=df2.merge(add,on=columns,how="left")
    del add
    return df2

def merge_value_count(df1,df2,col, name=None, divisor=None):
    if name is None:
        name = col[1]
    tmp = df1.groupby(col).size().reset_index().rename(columns = {0:'cnt'})
    df = tmp.pivot(index=col[0],columns=col[1],values='cnt').reset_index()
    if divisor is not None:
        df = df.apply(lambda x: [x.iloc[0], *(x.iloc[1:] / divisor[x.iloc[0]])], axis=1, result_type='broadcast')
    cname = [col[0]]
    for index in range(1,len(df.columns)):
        cname.append(str(name)+'_'+str(df.columns[index]))
    df.columns=cname
    df = df.fillna(0)
    df2 = df2.merge(df,on=str(col[0]),how='left')
    del df,df1
    return df2

In [7]:
import geohash
#Geo info
def putGeoInfo(df):
    geo = df['geo_code'].apply(lambda x: ((x, x) if pd.isna(x) else geohash.decode(x)))
    df['latitude'] = geo.apply(lambda x: x[0])
    df['longtitude'] = geo.apply(lambda x: x[1])

dfs = [operation_df, operation_test, transaction_df, transaction_test]


for df in dfs:
    putGeoInfo(df)

In [8]:
import sklearn.cluster
def putGeoCluster(df, test_df):
    kmeans = sklearn.cluster.KMeans(n_clusters=34, n_jobs=-1)
    kmeans.fit(df[['latitude', 'longtitude']].fillna(-1))
    df['geo_cluster'] = kmeans.predict(df[['latitude', 'longtitude']].fillna(-1))
    test_df['geo_cluster'] = kmeans.predict(test_df[['latitude', 'longtitude']].fillna(-1))
    
putGeoCluster(operation_df, operation_test)
putGeoCluster(transaction_df, transaction_test)

In [9]:
labels_dict = {}
for i, row in label.iterrows():
    labels_dict[row['UID']] = row['Tag']
    
def prepare_ratio_data(transaction_df, label, name):
    col = transaction_df.groupby('UID')[name]
    category_sum = dict()
    category_black_count = dict()
    def manage_categories(m):
        uid = m.name
        m = m.unique()
        for c_id in m:
            category_sum[c_id] = category_sum.get(c_id, 0) + 1
            if labels_dict[uid] == 1:
                category_black_count[c_id] = category_black_count.get(c_id, 0) + 1
    col.apply(manage_categories)
        
    return {m_id: category_black_count.get(m_id, 0) / m_count for m_id, m_count in category_sum.items()}
        


In [10]:

black_ratio = {
    'merchant': prepare_ratio_data(transaction_df, label, 'merchant'),
    'amt_src1': prepare_ratio_data(transaction_df, label, 'amt_src1'),
    'trans_device2': prepare_ratio_data(transaction_df, label, 'device2'),
    'op_device2': prepare_ratio_data(operation_df, label, 'device2'),
#     'acc_id2': prepare_ratio_data(transaction_df, label, 'acc_id2'),
    'acc_id3': prepare_ratio_data(transaction_df, label, 'acc_id3'),
    'op_geo': prepare_ratio_data(operation_df, label, 'geo_cluster'),
    'trans_geo': prepare_ratio_data(transaction_df, label, 'geo_cluster'),
}
black_avg = {
    k: np.mean([ratio.get(m, 0) for m in ratio.keys()]) for k, ratio in black_ratio.items()
}

In [11]:

def get_black_ratio(df, df_name, out_name):
    df['%s_black_ratio'%out_name] = df[df_name].apply(lambda x: black_ratio[out_name].get(x, black_avg[out_name]))
    tmp = df.groupby('UID')['%s_black_ratio'%out_name].agg([max, min, np.mean]).reset_index()
    tmp = pd.merge(tmp, df.groupby('UID')['%s_black_ratio'%out_name].skew().fillna(0), on='UID')
    tmp.columns = ['UID', 'max_%s_black_ratio'%out_name, "min_%s_black_ratio"%out_name, "avg_%s_black_ratio"%out_name, 'skew_%s_black_ratio'%out_name]
    return tmp


In [12]:
def getPopularityCount(df, df_name, out_name):
    categories = df[df_name].unique()
    counts = {c: 0 for c in categories}
    counts[np.nan] = np.nan
    groups = df[['UID', df_name]].groupby('UID')
    for uid, frame in groups:
        for val in frame[df_name].unique():
            counts[val] += 1
    counts_list = [counts[v] for v in df[df_name]]
    df[out_name] = counts_list
    res = df.groupby('UID')[out_name].agg([max,min,np.mean]).reset_index()
    res = pd.merge(res, df.groupby('UID')[out_name].skew().fillna(0), on='UID')
    res.columns = ['UID', '%s_max'%out_name, '%s_min'%out_name, '%s_avg'%out_name, '%s_skew'%out_name]
    return res

In [13]:
(transaction_test.groupby('day').size()[:10],
 transaction_df.groupby('day').size()[:10])

(day
 1     2620
 2     3092
 3     3091
 4     2869
 5     3473
 6     8619
 7     6821
 8     3637
 9     2580
 10    2731
 dtype: int64, day
 1     17767
 2     11241
 3      3086
 4      2826
 5      4676
 6      3309
 7      3437
 8     20130
 9     10085
 10     3304
 dtype: int64)

In [14]:
train_weekday = lambda x: (x + 5) % 7
test_weekday = lambda x: x % 7

In [15]:
def add_week_day_features(dataframe, weekday_func):
    days = dataframe['day'].apply(lambda x: weekday_func(x))
    days = days.replace(0, 7)
    dataframe['day_in_week'] = days
    return dataframe
operation_df = add_week_day_features(operation_df, train_weekday)
transaction_df = add_week_day_features(transaction_df, train_weekday)
operation_test = add_week_day_features(operation_test, test_weekday)
transaction_test = add_week_day_features(transaction_test, test_weekday)

In [16]:
def get_col_relate_info(data, col):
    df = pd.DataFrame(data.groupby(col)['UID'].nunique())
    df.columns = ['cnt_uid_' + col]
    df['min_day_' + col] = data.groupby(col)['day'].min()
    df['max_day_' + col] = data.groupby(col)['day'].max()
    df['gap_day_' + col] = df['max_day_' + col] - df['min_day_' + col]
    
    df['min_longtitude_' + col] = data.groupby(col)['longtitude'].min()
    df['max_longtitude_' + col] = data.groupby(col)['longtitude'].max()
    df['gap_longtitude_' + col] = df['max_longtitude_' + col] - df['min_longtitude_' + col]

    df['min_latitude_' + col] = data.groupby(col)['latitude'].min()
    df['max_latitude_' + col] = data.groupby(col)['latitude'].max()
    df['gap_latitude_' + col] = df['max_latitude_' + col] - df['min_latitude_' + col]

    if 'trans_ratio' in data.columns:
        df['min_trans_ratio_' + col] = data.groupby(col)['trans_ratio'].min()
        df['max_trans_ratio_' + col] = data.groupby(col)['trans_ratio'].max()
        df['gap_trans_ratio_' + col] = df['max_trans_ratio_' + col] - df['min_trans_ratio_' + col]

    df = df.reset_index()
    return df

In [17]:
[e for e in['acc_id1', 'acc_id2', 'acc_id3', 'amt_src1', 'amt_src2', 'code1', 'code2',
                  'device_code1', 'device_code2', 'device_code3', 'geo_code', 'ip1', 'ip1_sub',
                'mac1', 'market_code', 'merchant'] if e not in transaction_df.columns]

[]

In [18]:
def gap(x):
    valid = x.dropna()
    if not valid.empty:
        return max(valid) - min(valid)
    else:
        return 0

def get_op_stats_fea(operation_df):
    op_counts = operation_df.groupby('UID').size()
    
    #op_day
    op_fea = operation_df[['UID']].drop_duplicates()
    tmp = operation_df.groupby('UID')['day'].agg([max,min,gap]).reset_index()
    tmp.columns=['UID','op_day_max','op_day_min', 'op_day_gap']
    op_fea = pd.merge(op_fea,tmp,on='UID',how='left')
    
    
    tmp = operation_df.groupby('UID')['latitude'].agg([max,min,gap]).reset_index()
    tmp.columns = ['UID', 'op_latitude_max', 'op_latitude_min', 'op_latitude_gap']
    op_fea = pd.merge(op_fea, tmp, on='UID', how='left')
    tmp = operation_df.groupby('UID')['longtitude'].agg([max,min,gap]).reset_index()
    tmp.columns = ['UID', 'op_longtitude_max', 'op_longtitude_min', 'op_longtitude_gap']
    op_fea = pd.merge(op_fea, tmp, on='UID', how='left')
    #op_mode count
    op_fea = merge_count(operation_df,op_fea,'UID','mode','op_cnt')
    op_fea = merge_nunique(operation_df,op_fea,'UID','mode','op_mode_nunique')
    
    #op_time
    operation_df['op_hour'] = operation_df['time'].apply(lambda x:int(x.split(':')[0]))
        
    #op_os
    for col in ['os','version','device1','device2','device_code1','device_code2','mac1','ip1','ip2','device_code3','mac2','wifi','ip1_sub','ip2_sub','geo_code']:
        op_fea = merge_nunique(operation_df,op_fea,'UID',col,'op_'+col+'_nunique')
        op_fea = merge_notnull_count(operation_df,op_fea,'UID',col,'op_'+col+'_notnull_cnt')
        
        
    for col in [ 'version', 'device_code1', 'device_code2', 'device_code3', 'ip1', 'ip1_sub', 'ip2',
                  'ip2_sub', 'mac1', 'mac2', 'wifi', 'mode']:
        tmp = get_col_relate_info(operation_df, col)
        tmp = pd.merge(operation_df[['UID', col]].drop_duplicates(), tmp, on=col, how='left')
        for c in ['day', 'latitude', 'longtitude']:
            for m in ['min', 'max', 'gap']:
                stats = tmp.groupby('UID')['%s_%s_%s'%(m, c, col)].agg([max,min,np.mean]).reset_index()
                stats = pd.merge(stats, tmp.groupby('UID')['%s_%s_%s'%(m, c, col)].skew().fillna(0), on='UID')
                stats.columns=['UID','op_%s_%s_%s_max'%(m, c, col), 'op_%s_%s_%s_min'%(m, c, col), 'op_%s_%s_%s_mean'%(m, c, col), 'op_%s_%s_%s_skew'%(m, c, col)]
                op_fea = pd.merge(op_fea, stats, on='UID', how='left')
        
    return op_fea

def get_trans_stats_fea(transaction_df):
    trans_counts = transaction_df.groupby('UID').size()
    
    trans_fea = transaction_df[['UID']].drop_duplicates()
        
    tmp = transaction_df.groupby('UID')['day'].agg([max,min,gap]).reset_index()
    tmp.columns=['UID','trans_day_max','trans_day_min','trans_day_gap']
    trans_fea = pd.merge(trans_fea,tmp,on='UID',how='left')
    
    tmp = transaction_df.groupby('UID')['trans_amt'].agg([max,min,np.mean,gap]).reset_index()
    tmp = pd.merge(tmp, transaction_df.groupby('UID')['trans_amt'].skew().fillna(0), on='UID')
    tmp.columns=['UID','trans_amt_max','trans_amt_min','trans_amt_mean', 'trans_amt_gap', 'trans_amt_skew']
    trans_fea = pd.merge(trans_fea,tmp,on='UID',how='left')
    
    tmp = transaction_df.groupby('UID')['bal'].agg([max,min,np.mean,gap]).reset_index()
    tmp = pd.merge(tmp, transaction_df.groupby('UID')['bal'].skew().fillna(0), on='UID')
    tmp.columns=['UID','bal_max','bal_min','bal_mean', 'bal_gap', 'bal_skew']
    trans_fea = pd.merge(trans_fea,tmp,on='UID',how='left')
    
    tmp = transaction_df.groupby('UID')['latitude'].agg([max,min,gap]).reset_index()
    tmp.columns = ['UID', 'trans_latitude_max', 'trans_latitude_min', 'trans_latitude_gap']
    trans_fea = pd.merge(trans_fea, tmp, on='UID', how='left')
    tmp = transaction_df.groupby('UID')['longtitude'].agg([max,min,gap]).reset_index()
    tmp.columns = ['UID', 'trans_longtitude_max', 'trans_longtitude_min', 'trans_longtitude_gap']
    trans_fea = pd.merge(trans_fea, tmp, on='UID', how='left')

    transaction_df['trans_ratio'] = transaction_df['trans_amt'] / transaction_df['bal']
    tmp = transaction_df.groupby('UID')['trans_ratio'].agg([max,min,np.mean,gap]).reset_index()
    tmp = pd.merge(tmp, transaction_df.groupby('UID')['trans_ratio'].skew().fillna(0), on='UID')
    tmp.columns=['UID','trans_ratio_max','trans_ratio_min','trans_ratio_mean', 'trans_ratio_gap', 'trans_ratio_skew']
    trans_fea = pd.merge(trans_fea,tmp,on='UID',how='left')
    
    #trans time
    transaction_df['trans_hour'] = transaction_df['time'].apply(lambda x:int(x.split(':')[0]))
    
    for col in ['trans_type1','merchant','code1','code2','acc_id1','device_code1','device_code2','device_code3','device1','device2','mac1','ip1','acc_id2','acc_id3','market_code','ip1_sub','geo_code']:
        trans_fea = merge_nunique(transaction_df,trans_fea,'UID',col,'trans_'+col+'_nunique')
        trans_fea = merge_notnull_count(transaction_df,trans_fea,'UID',col,'trans_'+col+'_notnull_cnt')
    for col in ['trans_type2','market_type']:
        trans_fea = merge_nunique(transaction_df,trans_fea,'UID',col,'trans_'+col+'_nunique')
        trans_fea = merge_notnull_count(transaction_df,trans_fea,'UID',col,'trans_'+col+'_notnull_cnt')
        
        
    for col in ['acc_id1', 'acc_id2', 'acc_id3', 'amt_src1', 'amt_src2', 'code1', 'code2',
                'device_code1', 'device_code2', 'device_code3', 'geo_code', 'ip1', 'ip1_sub',
                'mac1', 'market_code', 'merchant']:
        tmp = get_col_relate_info(transaction_df, col)
        tmp = pd.merge(transaction_df[['UID', col]].drop_duplicates(), tmp, on=col, how='left')
        for c in ['day', 'latitude', 'longtitude']:
            for m in ['min', 'max', 'gap']:
                stats = tmp.groupby('UID')['%s_%s_%s'%(m, c, col)].agg([max,min,np.mean]).reset_index()
                stats = pd.merge(stats, tmp.groupby('UID')['%s_%s_%s'%(m, c, col)].skew().fillna(0), on='UID')
                stats.columns=['UID','trans_%s_%s_%s_max'%(m, c, col), 'trans_%s_%s_%s_min'%(m, c, col), 'trans_%s_%s_%s_mean'%(m, c, col), 'trans_%s_%s_%s_skew'%(m, c, col)]
                trans_fea = pd.merge(trans_fea, stats, on='UID', how='left')
    return trans_fea


In [19]:
def get_trans_counted_features(transaction_df):
    trans_counts = transaction_df.groupby('UID').size()
    trans_fea = transaction_df[['UID']].drop_duplicates()
    #trans_channel
    trans_fea = merge_count(transaction_df,trans_fea,'UID','channel','trans_cnt')
    trans_fea = merge_nunique(transaction_df,trans_fea,'UID','channel','trans_channel_nunique')
    transaction_df['trans_hour'] = transaction_df['time'].apply(lambda x:int(x.split(':')[0]))
#     trans_fea = merge_value_count(transaction_df,trans_fea,['UID','trans_hour'], divisor=trans_counts)
    for col in ['market_type']:
        trans_fea = merge_value_count(transaction_df,trans_fea,['UID',col], 'trans_%s'%col, divisor=trans_counts)
        
    trans_fea = merge_value_count(transaction_df,trans_fea,['UID','day_in_week'], 'trans_week_day')

    trans_fea = merge_nunique(transaction_df,trans_fea,'UID','geo_cluster','trans_geo_nunique')
    
    trans_fea['trans_on_weekend'] = trans_fea[['trans_week_day_%d'%i for i in range(6, 8)]].sum(axis=1)
    trans_fea['trans_on_weekday'] = trans_fea[['trans_week_day_%d'%i for i in range(1, 6)]].sum(axis=1)
    
    trans_fea['trans_weekday_weekend_ratio'] = trans_fea['trans_on_weekday'] / trans_fea['trans_on_weekend']
    
    pop_count = [
        ('device2', 'trans_device2_pop')
    ]
    for df_name, out_name in pop_count:
        pop = getPopularityCount(transaction_df, df_name, out_name)
        trans_fea = pd.merge(trans_fea,pop,on='UID',how='left')
    
    ratio_list = [
        ('merchant', 'merchant'),
        ('amt_src1', 'amt_src1'),
        ('device2', 'trans_device2'),
#         ('acc_id2', 'acc_id2'),
        ('acc_id3', 'acc_id3'),
        ('geo_cluster', 'trans_geo')
    ]
    
    for df_name, out_name in ratio_list:
        ratios = get_black_ratio(transaction_df, df_name, out_name)
        trans_fea = trans_fea.merge(ratios, on='UID', how='left')
    return trans_fea
    
    
def get_op_counted_features(operation_df):
    op_counts = operation_df.groupby('UID').size()
    
    op_fea = operation_df[['UID']].drop_duplicates()
#     op_fea = merge_value_count(operation_df,op_fea,['UID','day'],'op_days', divisor=op_counts)
    #success count
    op_fea = merge_count(operation_df[operation_df.success==0],op_fea,'UID','mode','op_fail_cnt')
    op_fea = merge_count(operation_df[operation_df.success==1],op_fea,'UID','mode','op_success_cnt')
    op_fea['op_fail_cnt'] = op_fea['op_fail_cnt'].fillna(0)
    op_fea['op_success_cnt'] = op_fea['op_success_cnt'].fillna(0)
#     op_fea = merge_value_count(operation_df,op_fea,['UID','op_hour'], divisor=op_counts) 
    op_fea = merge_value_count(operation_df,op_fea,['UID','day_in_week'], 'op_week_day') 

    op_fea['op_on_weekend'] = op_fea[['op_week_day_%d'%i for i in range(6, 8)]].sum(axis=1)
    op_fea['op_on_weekday'] = op_fea[['op_week_day_%d'%i for i in range(1, 6)]].sum(axis=1)
    
    op_fea['op_weekday_weekend_ratio'] = op_fea['op_on_weekday'] / op_fea['op_on_weekend']
    
    op_fea = merge_nunique(operation_df,op_fea,'UID','geo_cluster','op_geo_nunique')
    
    pop_count = [
        ('wifi', 'op_wifi_pop'),
        ('os', 'op_os_pop'),
        ('version', 'op_version_pop'),
        ('device2', 'op_device2_pop')
    ]
    for df_name, out_name in pop_count:
        pop = getPopularityCount(operation_df, df_name, out_name)
        op_fea = pd.merge(op_fea,pop,on='UID',how='left')
    
    ratio_list = [
        ('device2', 'op_device2'),
        ('geo_cluster', 'op_geo')
    ]
    
    for df_name, out_name in ratio_list:
        ratios = get_black_ratio(operation_df, df_name, out_name)
        op_fea = op_fea.merge(ratios, on='UID', how='left')
    
    return op_fea

In [20]:
#Stats
operation_features_stats = get_op_stats_fea(operation_df)
transaction_features_stats = get_trans_stats_fea(transaction_df)

In [21]:
operation_test_stats = get_op_stats_fea(operation_test)
transaction_test_stats = get_trans_stats_fea(transaction_test)

In [22]:
#counted data
operation_features_counted = get_op_counted_features(operation_df)
transaction_features_counted = get_trans_counted_features(transaction_df)

In [23]:
operation_test_counted = get_op_counted_features(operation_test)
transaction_test_counted = get_trans_counted_features(transaction_test)

In [24]:
train_features = operation_features_stats.merge(transaction_features_stats, on='UID', how='outer')
train_features = train_features.merge(operation_features_counted, on='UID', how='outer')
train_features = train_features.merge(transaction_features_counted, on='UID', how='outer')
train_data = train_features.merge(label,on='UID',how='left')
train_data = train_data.drop('UID', axis=1)
test_features = operation_test_stats.merge(transaction_test_stats, on='UID', how='outer').sort_values(['UID'])
test_features = test_features.merge(operation_test_counted, on='UID', how='outer')
test_features = test_features.merge(transaction_test_counted, on='UID', how='outer').sort_values(['UID'])

UIDs = test_features['UID']
test_features = test_features.drop('UID', axis=1)

# columns_to_drop = [col for col in train_data.columns if col not in test_features.columns and col != 'Tag']
# train_data = train_data.drop(columns_to_drop, axis=1)
# columns_to_drop = [col for col in test_features.columns if col not in train_data.columns and col != 'Tag']
# test_features = test_features.drop(columns_to_drop, axis=1)

#Fill nan with average
train_data = train_data.fillna(train_data.mean())
test_features = test_features.fillna(train_data.drop('Tag', axis=1).mean())

In [25]:
import sklearn.preprocessing
def preprocess(x):
    scaler = sklearn.preprocessing.MinMaxScaler()
    return scaler.fit_transform(x)

In [26]:
def tpr_scorer(estimator, x, y_true):
    y_predict = estimator.predict_proba(x)[:, 1]
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3

In [27]:
import sklearn.model_selection
import sklearn.metrics
import lightgbm as lgb
import sklearn.ensemble
import xgboost as xgb

def cv(x, y, params={}, splits=5):
    clf = lgb.LGBMClassifier(**params)
#     clf = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=-1)
#     clf = xgb.XGBClassifier(**params)
    kfold = sklearn.model_selection.StratifiedKFold(splits, shuffle=True)
    cv_score = sklearn.model_selection.cross_validate(clf, x, y, cv=kfold, scoring={
        'tpr': tpr_scorer,
        'accuracy': 'accuracy',
        'f1': 'f1_micro',
        'roc_auc': 'roc_auc'
    }, return_train_score=True)
    return cv_score

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [28]:
params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        "num_leaves": 200,
        "max_depth": -1,
        "learning_rate": 0.1,
        'min_child_samples': 100,
        'n_estimators': 2000,
        'learning_rate': 0.05,
        'scale_pos_weight': float(label[label.Tag==0].shape[0]) / label[label.Tag==1].shape[0],
        'boost_from_average': True,
        'min_child_weight': 1e-3,
        "subsample_for_bin": 20000,
        'max_bin': 512,
        "metric": 'auc',
        'reg_alpha': 3,
        'reg_lambda': 5,
        'subsample': 0.9,
        'colsample_bytree':0.7, 
        'subsample_freq': 1,
        'n_jobs': -1,
}
# params = {
#         'booster':'gbtree',
#         'objective':'binary:logistic',
#         'stratified':True,
#         'max_depth':8,
#         'min_child_weight':1,
#         'gamma':3,
#         'subsample':0.8,#0.7
#         'colsample_bytree':0.6, 
#         'reg_lambda':3, 
#         'eta':0.05,
#         'seed':20,
#         'silent':1,
#         'eval_metric':'auc',
#         'n_jobs': 12,
#         'n_estimators': 400,
#         'early_stopping_round': 200,
#         'scale_pos_weight': float(label[label.Tag==0].shape[0]) / label[label.Tag==1].shape[0],
#         'verbosity': 2
# }

def run_cross_validation(x, y):
    cv_result = cv(x, y, params=params, splits=5)
    for scorer, score in cv_result.items():
        print('%s: %s' % (scorer, score))
        print('Average %s: %f' % (scorer, score.mean()))

In [29]:
run_cross_validation(train_data.drop('Tag', axis=1).fillna(train_data.drop('Tag', axis=1).mean()).values, train_data.loc[:, 'Tag'].values)

fit_time: [40.73458624 37.43092299 29.4991889  29.66712093 29.93988514]
Average fit_time: 33.454341
score_time: [0.43032384 0.3479681  0.37895989 0.3469398  0.51744175]
Average score_time: 0.404327
test_tpr: [0.85344224 0.83418903 0.85495916 0.84784131 0.85670945]
Average test_tpr: 0.849428
train_tpr: [1. 1. 1. 1. 1.]
Average train_tpr: 1.000000
test_accuracy: [0.97434253 0.9712957  0.97658756 0.9748236  0.97530072]
Average test_accuracy: 0.974470
train_accuracy: [0.99959909 0.99987973 0.99983963 0.99963918 0.99971937]
Average train_accuracy: 0.999735
test_f1: [0.97434253 0.9712957  0.97658756 0.9748236  0.97530072]
Average test_f1: 0.974470
train_f1: [0.99959909 0.99987973 0.99983963 0.99963918 0.99971937]
Average train_f1: 0.999735
test_roc_auc: [0.98957374 0.99174498 0.99324407 0.99124626 0.99289686]
Average test_roc_auc: 0.991741
train_roc_auc: [0.99999995 1.         0.99999999 0.99999996 0.99999997]
Average train_roc_auc: 1.000000


In [30]:
def train(x, y, params={}):
    clf = lgb.LGBMClassifier(**params)
#     clf = xgb.XGBClassifier(**params)
    clf.fit(x, y)
    return clf

In [31]:
model = train(train_data.drop('Tag', axis=1).values, train_data.loc[:, 'Tag'].values, params=params)

In [32]:
result = model.predict_proba(test_features.values)[:, 1]

In [33]:
(result > 0.5).sum() / len(result)

0.09250592986729919

In [34]:
tpr_scorer(model, train_data.drop('Tag', axis=1).fillna(train_data.drop('Tag', axis=1).mean()).values, train_data.loc[:, 'Tag'].values)

1.0

In [35]:
result_frame = pd.DataFrame()
result_frame['UID'] = UIDs
result_frame['Tag'] = result

In [36]:

result_frame.to_csv(res_path + 'res.csv', index=False)

In [37]:
import matplotlib.pyplot as pyplot

In [41]:
importances = model.feature_importances_
for fi in [(train_data.drop('Tag', axis=1).columns[i], importances[i]) for i in range(len(importances))]:
    print(fi)

('op_day_max', 80)
('op_day_min', 104)
('op_day_gap', 52)
('op_latitude_max', 24)
('op_latitude_min', 26)
('op_latitude_gap', 23)
('op_longtitude_max', 12)
('op_longtitude_min', 24)
('op_longtitude_gap', 23)
('op_cnt', 285)
('op_mode_nunique', 119)
('op_os_nunique', 7)
('op_os_notnull_cnt', 85)
('op_version_nunique', 4)
('op_version_notnull_cnt', 108)
('op_device1_nunique', 10)
('op_device1_notnull_cnt', 38)
('op_device2_nunique', 5)
('op_device2_notnull_cnt', 155)
('op_device_code1_nunique', 9)
('op_device_code1_notnull_cnt', 122)
('op_device_code2_nunique', 34)
('op_device_code2_notnull_cnt', 175)
('op_mac1_nunique', 3)
('op_mac1_notnull_cnt', 87)
('op_ip1_nunique', 54)
('op_ip1_notnull_cnt', 79)
('op_ip2_nunique', 8)
('op_ip2_notnull_cnt', 26)
('op_device_code3_nunique', 3)
('op_device_code3_notnull_cnt', 45)
('op_mac2_nunique', 7)
('op_mac2_notnull_cnt', 201)
('op_wifi_nunique', 14)
('op_wifi_notnull_cnt', 356)
('op_ip1_sub_nunique', 96)
('op_ip1_sub_notnull_cnt', 28)
('op_ip2_sub_

In [40]:
train_data.shape

(31179, 1188)